In [1]:
import pandas as pd
import numpy as np
import json
import random

In [2]:
train_df = pd.read_csv("../data/v1/train.csv")
test_df = pd.read_csv("../data/v1/test.csv")
meta_df = pd.read_csv("../data/item_metadata_filtered.csv")
with open("../data/id_mappings.json") as f:
    id_map = json.load(f)

In [3]:
# User id and asin to int id mapping
asin_to_id = {asin: int(item_id) for asin, item_id in id_map["item_mapping"].items()} # asin (str) -> item_id (int)
item_id_to_asin = {int(item_id): asin for item_id, asin in id_map["item_reverse_mapping"].items()} # item_id (int) -> asin (str)

# Create a lookup dictionary for true items from the test set
true_items_map = pd.Series(test_df.item_id.values, index=test_df.user_id).to_dict()

In [4]:
all_categories = sorted(meta_df["main_category"].dropna().unique().tolist())
category_options = ["Any"] + all_categories

In [5]:
# Preprocess metadata
meta_df["title"] = meta_df["title"].fillna("")
meta_df["store"] = meta_df["store"].fillna("")
meta_df["description"] = meta_df["description"].fillna("")
meta_df["average_rating"] = meta_df["average_rating"].fillna("")
meta_df["price"] = meta_df["price"].fillna("")
meta_df["image_urls"] = meta_df["image_urls"].fillna("[]")

# Index by ASIN for lookup
meta_df = meta_df.set_index("parent_asin")

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# --- TF-IDF ---
all_items = sorted(asin_to_id.keys())  # all ASINs
texts = [
    meta_df.loc[asin]["title"] + " " +
    meta_df.loc[asin]["store"] + " " +
    meta_df.loc[asin]["description"]
    if asin in meta_df.index else ""
    for asin in all_items
]

In [8]:
vectorizer = TfidfVectorizer(max_features=10_000)
tfidf_matrix = vectorizer.fit_transform(texts)

# Basic TD-IDF
with de-duplication of results

In [9]:
def build_user_profile(user_id):
    user_ratings = train_df[train_df["user_id"] == user_id]
    indices = user_ratings["item_id"].values
    ratings = user_ratings["rating"].values
    item_vectors = tfidf_matrix[indices]
    weighted = item_vectors.multiply(ratings[:, None])
    profile = weighted.mean(axis=0)
    return np.asarray(profile)

In [19]:
def recommend_for_user_v1(user_id, top_k=10):
    profile = build_user_profile(user_id)
    scores = cosine_similarity(profile, tfidf_matrix).flatten()
    # Remove previously rated items
    seen_items = set(train_df[train_df["user_id"] == user_id]["item_id"])
    recs = [i for i in scores.argsort()[::-1] if i not in seen_items]
    return recs[:top_k]

In [11]:
def average_precision_at_k(recommendations, true_item_id, k=10):
    if true_item_id in recommendations[:k]:
        # Find the rank (position) of the true item, adding 1 for 1-based ranking
        rank = recommendations[:k].index(true_item_id) + 1
        return 1.0 / rank
    return 0.0

In [12]:
def pick_random_user_simple(min_interactions, max_interactions):
    counts = train_df["user_id"].value_counts()
    if max_interactions <= 0:
        valid = counts[counts >= min_interactions].index.tolist()
    else:
        valid = counts[min_interactions <= counts <= max_interactions].index.tolist()
    return random.choice(valid), len(valid), len(counts) if valid else None

In [13]:
from tqdm import tqdm

In [14]:
def map_at_10(validation_df, recommend_func, k=10):
    ap_scores = []
    for _, row in tqdm(validation_df.iterrows(), total=len(validation_df), desc="Calculating MAP@10"):
        user_id = row["user_id"]
        true_item_id = row["item_id"]
        recommendations = recommend_func(user_id, top_k=k)
        ap = average_precision_at_k(recommendations, true_item_id, k)
        ap_scores.append(ap)
    return np.mean(ap_scores) if ap_scores else 0.0


In [16]:
def map_at_10_sampled(validation_df, recommend_func, k=10, sample_size=10_000, random_state=42):
    unique_users = validation_df["user_id"].unique()
    rng = np.random.default_rng(random_state)
    sampled_users = rng.choice(unique_users, size=min(sample_size, len(unique_users)), replace=False)
    sampled_df = validation_df[validation_df["user_id"].isin(sampled_users)]

    ap_scores = []
    for _, row in tqdm(sampled_df.iterrows(), total=len(sampled_df), desc="Calculating MAP@10 (sampled)"):
        user_id = row["user_id"]
        true_item_id = row["item_id"]
        recommendations = recommend_func(user_id, top_k=k)
        ap = average_precision_at_k(recommendations, true_item_id, k)
        ap_scores.append(ap)
    return np.mean(ap_scores) if ap_scores else 0.0

In [17]:
rec_v1 = map_at_10_sampled(test_df, recommend_for_user_v1, k=10, sample_size=1_000)

Calculating MAP@10 (sampled): 100%|██████████| 1000/1000 [01:21<00:00, 12.20it/s]


In [18]:
print(f"MAP@10 for TD-IDF v1: {rec_v1:.4f}")

np.float64(0.010917857142857142)

# TD-IDF v2
with boosting most popular items from user's main category

In [20]:
def get_user_main_category(user_id):
    user_items = train_df[train_df["user_id"] == user_id]["item_id"]
    asins = [item_id_to_asin[i] for i in user_items if i in item_id_to_asin]
    categories = meta_df.loc[asins, "main_category"].dropna()
    if categories.empty:
        return None
    return categories.value_counts().idxmax()

In [21]:
def get_category_popularity(category):
    items_in_cat = meta_df[meta_df["main_category"] == category].index
    item_ids = [asin_to_id[a] for a in items_in_cat if a in asin_to_id]
    pop = train_df[train_df["item_id"].isin(item_ids)]["item_id"].value_counts()
    return pop

In [22]:
def recommend_for_user_v2(user_id, top_k=10, alpha=0.7):
    profile = build_user_profile(user_id)
    scores = cosine_similarity(profile, tfidf_matrix).flatten()
    seen_items = set(train_df[train_df["user_id"] == user_id]["item_id"])
    main_cat = get_user_main_category(user_id)
    if main_cat is None:
        # fallback to original recommend
        recs = [i for i in scores.argsort()[::-1] if i not in seen_items]
        return recs[:top_k]
    pop = get_category_popularity(main_cat)
    pop_scores = np.zeros_like(scores)
    for item_id, count in pop.items():
        pop_scores[item_id] = count
    # Normalize both scores
    if pop_scores.max() > 0:
        pop_scores = pop_scores / pop_scores.max()
    if scores.max() > 0:
        scores = scores / scores.max()
    # Combine: alpha * tfidf + (1-alpha) * popularity
    combined = alpha * scores + (1 - alpha) * pop_scores
    recs = [i for i in combined.argsort()[::-1] if i not in seen_items]
    return recs[:top_k]

In [23]:
rec_v2 = map_at_10_sampled(test_df, recommend_for_user_v2, k=10, sample_size=1_000)

Calculating MAP@10 (sampled): 100%|██████████| 1000/1000 [04:16<00:00,  3.90it/s]


In [27]:
print(f"MAP@10 for TD-IDF v2: {rec_v2:.4f}")

MAP@10 for TD-IDF v2: 0.0239


In [31]:
len(train_df["user_id"].unique()), len(test_df["user_id"].unique())

(868218, 868218)